<a href="https://colab.research.google.com/github/czelijm/SNR_18Z/blob/master/SNR_Zadanie2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Podpinanie dysku google, importowanie bibliotek oraz definicje funkcji



In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd /content/drive/WorkingSpaceColab/SNR

/content/drive/WorkingSpaceColab/SNR


In [0]:
 ls -la

In [0]:
import os
from collections import Counter

import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from skimage import feature
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16
# from keras.preprocessing.image import load_img
from keras import layers
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation, InputLayer
from matplotlib import pyplot
from skimage.util import random_noise



Using TensorFlow backend.


In [0]:
LR_ADAMA = 0.0001

In [0]:
def check_numpy_set_exist(name):
    if os.path.isfile("dataX-" + name + ".npy") and os.path.isfile("dataY-" + name + ".npy"):
        return True
    return False

In [0]:
def load_numpy_set(name):
    x = np.load("dataX-" + name + ".npy")
    y = np.load("dataY-" + name + ".npy")
    return x, y

In [0]:
def make_id_class_dictionary(y_val):
    # make dictionary of classes
    y_val.sort()
    y_val = Counter(y_val).keys()

    class_id = {}
    temp_list_id = []
    count = 1

    for i in y_val:
        temp_list_id.append((i, count))
        count += 1

    for word, _id in temp_list_id:
        class_id[word] = _id
    print(class_id)

    return class_id

In [0]:
def cvt2_id_class_list(dictionary_class_id, word_list):
    # covert word list of class to id list of class
    y_teach = []
    for i in word_list:
        y_teach.append(dictionary_class_id[i])
    return y_teach

In [0]:
def cv2_id_class_as_vector(y_val_temp, num_of_class):
    y_val = []
    for i in y_val_temp:
        y = [0] * num_of_class
        y[i - 1] = 1
        y_val.append(y)
    np.asarray(y_val)
    return y_val

In [0]:

print("Dane trengowe wczytywanie rozpoczęte")
x_teach, y_teach = load_numpy_set("teach")
print("Dane trengowe wczytane")
  
print("Dane testowe wczytywanie rozpoczęte")
x_test, y_test_temp = load_numpy_set("test")
print("Dane testowe wczytane")
y_test_temp_copy = y_test_temp.copy()

# random Split Data set to teach
x_teach, x_val, y_teach_temp1, y_val_temp1 = train_test_split(x_teach, y_teach, test_size=0.3, random_state=42)
print("Dane podzielone na zbiór treningowy i walidacyjny")

# number of class in set
num_of_class = len(set(y_test_temp_copy))

# make idClasses
id_class_dictionary = make_id_class_dictionary(y_test_temp_copy)

# convert from wordList to numberList Of class
y_teach_temp2 = cvt2_id_class_list(id_class_dictionary, y_teach_temp1)
y_val_temp2 = cvt2_id_class_list(id_class_dictionary, y_val_temp1)

# get final representation of labels as vector ex. [0 0 0 0 ..... 0 1 0 0 0] - outputs of network
y_val = cv2_id_class_as_vector(y_val_temp2, num_of_class)
y_teach = cv2_id_class_as_vector(y_teach_temp2, num_of_class)


#zbiór testowy
y_test_tmp = cvt2_id_class_list(id_class_dictionary, y_test_temp)
y_test = cv2_id_class_as_vector(y_test_tmp, num_of_class)
#zmiana nazw zbioru testowego
x_test_temp=x_test

# 2a  zaimplementować dowolnie wybraną przez siebie głęboką sieć splotową - bez uczenia

## wagi zaincjalizowane z "imagenet":

In [0]:
filepath_not_trained="VGG_not_trained.hdf5"

In [0]:
vgg_conv_not_trained = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
model_not_trained = Sequential()
model_not_trained.add(vgg_conv_not_trained)
# # Dodawanie warstw
model_not_trained.add(layers.Flatten())
model_not_trained.add(layers.Dense(71, activation='softmax'))
# Schemat modelu
model_not_trained.summary()

model_not_trained.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])


print(model_not_trained.metrics_names)
print(model_not_trained.evaluate(x_val, np.asarray(y_val)))
print(model_not_trained.evaluate(x_test_temp, np.asarray(y_test)))
# results_not_trained = model_not_trained.fit(x_teach, np.asarray(y_teach), epochs=NUM_OF_EPOCH_not_trained, batch_size=BATH_SIZE_not_trained, validation_data=(x_val, np.asarray(y_val)),
#                     verbose=1,callbacks= cb_not_trained)




In [0]:
model_not_trained.save(filepath_not_trained)

##z losowo zaincjanizowanymi wagami:

In [0]:
vgg_conv_not_trained_rand = VGG16(weights=None, include_top=False, input_shape=(100, 100, 3))
model_not_trained_rand = Sequential()
model_not_trained_rand.add(vgg_conv_not_trained_rand)
# # Dodawanie warstw
model_not_trained_rand.add(layers.Flatten())
model_not_trained_rand.add(layers.Dense(71, activation='softmax'))
# Schemat modelu
model_not_trained_rand.summary()

model_not_trained_rand.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])


print(model_not_trained_rand.metrics_names)
print(model_not_trained_rand.evaluate(x_val, np.asarray(y_val)))
print(model_not_trained_rand.evaluate(x_test_temp, np.asarray(y_test)))
# results_not_trained = model_not_trained.fit(x_teach, np.asarray(y_teach), epochs=NUM_OF_EPOCH_not_trained, batch_size=BATH_SIZE_not_trained, validation_data=(x_val, np.asarray(y_val)),
#                     verbose=1,callbacks= cb_not_trained)


In [0]:
filepath_not_trained_rand="VGG_not_trained_rand.hdf5"

In [0]:
model_not_trained_rand.save(filepath_not_trained_rand)

# 2b  zaimplementować dowolnie wybraną przez siebie głęboką sieć splotową - z uczeniem

In [0]:
NUM_OF_EPOCH = 1
BATH_SIZE = 100
filepath="VGG_trained.hdf5"

In [0]:
# vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
vgg_conv = VGG16(weights=None, include_top=False, input_shape=(100, 100, 3))


# Tworzenie nowego modelu
model = Sequential()
model.add(vgg_conv)

# Dodawanie warstw
model.add(layers.Flatten())
model.add(layers.Dense(71, activation='softmax'))


# Schemat modelu
model.summary()

# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])

checkpoint = ModelCheckpoint(filepath, verbose=0, save_best_only=True)
cb = [checkpoint] 

results = model.fit(x_teach, np.asarray(y_teach), epochs=2, batch_size=BATH_SIZE, validation_data=(x_test, np.asarray(y_test)),
                    verbose=1,callbacks= cb)

In [0]:
best_vgg_model = load_model(filepath)

In [0]:
print ('Start evaluating')
evaluation_result = best_vgg_model.evaluate(x_test_temp, np.asarray(y_test))
print ('Finish evaluating:')
print(best_vgg_model.metrics_names)
print(evaluation_result)

# 2c wykonać zwielokrotnianie danych

##  Obroty

In [0]:
# obroty
datagennewnew_rot = ImageDataGenerator(rotation_range=90,
                                  fill_mode='nearest')
datagennewnew_rot.fit(x_teach)
for X_batch, y_batch in datagennewnew_rot.flow(x_teach, y_teach, batch_size=9):
    for i in range(0, 9):
        bgr = X_batch[i].reshape(100, 100, 3).astype('uint8')
        rgb = bgr[...,::-1]
        pyplot.imshow(rgb)
    pyplot.show()
    break

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
# Tworzenie nowego modelu
model = Sequential()
model.add(vgg_conv)

# Dodawanie warstw
model.add(layers.Flatten())
model.add(layers.Dense(71, activation='softmax'))

# Schemat modelu
model.summary()

In [0]:

#datagennewnew_rot = ImageDataGenerator(rotation_range=90,
#                                      fill_mode='nearest')
#datagennewnew_rot.fit(x_teach)
traind_data_generator = datagennewnew_rot.flow(x_teach, np.array(y_teach), batch_size=100)

# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])


results = model.fit_generator(traind_data_generator, steps_per_epoch=len(x_teach) / 100, epochs=2,validation_data=(x_test, np.asarray(y_test)), verbose=1)


In [0]:
file_path_2_save_file="VGG_based_model_rotated_images.hdf5"
model.save(file_path_2_save_file)

In [0]:
saveToFile = "rotated_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "rotated_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## Przesunięcie

In [0]:
#przesunięcia
datagennewnew_shift = ImageDataGenerator(width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
datagennewnew_shift.fit(x_teach)
for X_batch, y_batch in datagennewnew_shift.flow(x_teach, y_teach, batch_size=9):
    for i in range(0, 9):
        bgr = X_batch[i].reshape(100, 100, 3).astype('uint8')
        rgb = bgr[...,::-1]
        pyplot.imshow(rgb)
    pyplot.show()
    break

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
# Tworzenie nowego modelu
model = Sequential()
model.add(vgg_conv)

# Dodawanie warstw
model.add(layers.Flatten())
model.add(layers.Dense(71, activation='softmax'))

# Schemat modelu
model.summary()

In [0]:

traind_data_generator = datagennewnew_shift.flow(x_teach, np.array(y_teach), batch_size=100)

# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),#0.0001
              metrics=['categorical_accuracy'])

results = model.fit_generator(traind_data_generator, steps_per_epoch=len(x_teach) / 100, epochs=2,validation_data=(x_test, np.asarray(y_test)), verbose=1)


In [0]:
file_path_2_save_file="VGG_based_model_shifted_images.hdf5"
model.save(file_path_2_save_file)

In [0]:
saveToFile = "shifted_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "shifted_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## noise gaussian

In [0]:
#noise gaussian
datagennewnew_noise = ImageDataGenerator(fill_mode='nearest')
x_teach_noise = np.copy(x_teach)

for x in range(0, x_teach_noise.shape[0]):
    imgtmp = (x_teach_noise[x]).astype(np.float32) / 255.0
    imgtmp = random_noise(imgtmp,mode='gaussian',var=0.05**2)
    imgtmp=imgtmp * 255.0
    imgtmp= imgtmp.astype(np.int)
    x_teach_noise[x] = imgtmp

datagennewnew_noise.fit(x_teach_noise)
for X_batch, y_batch in datagennewnew_noise.flow(x_teach_noise, y_teach, batch_size=9):
    for i in range(0, 9):
        bgr = X_batch[i].reshape(100, 100, 3).astype('uint8')
        rgb = bgr[...,::-1]
        pyplot.imshow(rgb)
    pyplot.show()
    break  

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
# Tworzenie nowego modelu
model = Sequential()
model.add(vgg_conv)

# Dodawanie warstw
model.add(layers.Flatten())
model.add(layers.Dense(71, activation='softmax'))

# Schemat modelu
model.summary()

In [0]:

traind_data_generator = datagennewnew_noise.flow(x_teach, np.array(y_teach), batch_size=100)

# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=LR_ADAMA),
              metrics=['categorical_accuracy'])

results = model.fit_generator(traind_data_generator, steps_per_epoch=len(x_teach) / 100, epochs=2,validation_data=(x_test, np.asarray(y_test)), verbose=1)


In [0]:
path_file_gaussian_noise="best_VGG_based_model_gauss.hdf5"#przeuczona
model.save(path_file_gaussian_noise)

In [0]:
saveToFile = "gaussian_noise_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "gaussian_noise_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## noise s&p

In [0]:
#noise s&p
datagennewnew_noise = ImageDataGenerator(fill_mode='nearest')
x_teach_noise = np.copy(x_teach)

for x in range(0, x_teach_noise.shape[0]):
    imgtmp = (x_teach_noise[x]).astype(np.float32) / 255.0
    imgtmp = random_noise(imgtmp,mode='s&p')
    imgtmp=imgtmp * 255.0
    imgtmp= imgtmp.astype(np.int)
    x_teach_noise[x] = imgtmp

datagennewnew_noise.fit(x_teach_noise)
for X_batch, y_batch in datagennewnew_noise.flow(x_teach_noise, y_teach, batch_size=9):
    for i in range(0, 9):
        bgr = X_batch[i].reshape(100, 100, 3).astype('uint8')
        rgb = bgr[...,::-1]
        pyplot.imshow(rgb)
    pyplot.show()
    break  

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
# Tworzenie nowego modelu
model = Sequential()
model.add(vgg_conv)

# Dodawanie warstw
model.add(layers.Flatten())
model.add(layers.Dense(71, activation='softmax'))

# Schemat modelu
model.summary()

In [0]:
traind_data_generator = datagennewnew_noise.flow(x_teach, np.array(y_teach), batch_size=100)

# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=LR_ADAMA),#125
              metrics=['categorical_accuracy'])

results = model.fit_generator(traind_data_generator, steps_per_epoch=len(x_teach) / 100, epochs=2,validation_data=(x_test, np.asarray(y_test)), verbose=1)


In [0]:
path_file_sp_noise="VGG_based_model_sp.hdf5"#przeuczona
model.save(path_file_sp_noise)

In [0]:
saveToFile = "sp_noise_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "sp_noise_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## noise speckle

In [0]:
#noise s&p
datagennewnew_noise = ImageDataGenerator(fill_mode='nearest')
x_teach_noise = np.copy(x_teach)

for x in range(0, x_teach_noise.shape[0]):
    imgtmp = (x_teach_noise[x]).astype(np.float32) / 255.0
    imgtmp = random_noise(imgtmp,mode='speckle')
    imgtmp=imgtmp * 255.0
    imgtmp= imgtmp.astype(np.int)
    x_teach_noise[x] = imgtmp

datagennewnew_noise.fit(x_teach_noise)
for X_batch, y_batch in datagennewnew_noise.flow(x_teach_noise, y_teach, batch_size=9):
    for i in range(0, 9):
        bgr = X_batch[i].reshape(100, 100, 3).astype('uint8')
        rgb = bgr[...,::-1]
        pyplot.imshow(rgb)
    pyplot.show()
    break  

In [0]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
# Tworzenie nowego modelu
model = Sequential()
model.add(vgg_conv)

# Dodawanie warstw
model.add(layers.Flatten())
model.add(layers.Dense(71, activation='softmax'))

# Schemat modelu
model.summary()

In [0]:
traind_data_generator = datagennewnew_noise.flow(x_teach, np.array(y_teach), batch_size=100)

# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])

results = model.fit_generator(traind_data_generator, steps_per_epoch=len(x_teach) / 100, epochs=2,validation_data=(x_test, np.asarray(y_test)), verbose=1)


In [0]:
path_file_speckle_noise="VGG_based_model_speckle.hdf5"#przeuczona
model.save(path_file_speckle_noise)

In [0]:
saveToFile = "speckle_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "speckle_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

#2d  różne metody   regularyzacji: ​ dropout ,wczesne zatrzymanie uczenia, wpływ przesłonięcia  obrazów 
 

## Rozmiary deskryptora

In [0]:
#tworzenie struktury sieci
model = Sequential()

model.add(layers.InputLayer((100, 100, 3)))
model.add(layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv1'))
model.add( layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv1'))
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv1'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv2'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv1'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv1'))
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv2'))


# na tej warstwie trzeba dokonać modyfikować
model.add( layers.Convolution2D(256, (6, 6),
                         activation='relu',
                         padding='same',
                         name='block5_conv3'))

# trzy różne rozmairy deskryptora zmieniamy w tej warstwie powyżej "block5_conv3"
# model.add(layers.Convolution2D(64, (3, 3),
#                          activation='relu',
#                          padding='same',
#                          name='block5_conv3'))
# model.add(layers.Convolution2D(512, (5, 5),
#                          activation='relu',
#                          padding='same',
#                          name='block5_conv3'))
# model.add(layers.Convolution2D(512, (3, 3),
#                          activation='relu',
#                          padding='same',
#                          name='block5_conv3'))
###########################################



model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.add(layers.Flatten())

model.add(layers.Dense(71, activation='softmax'))


model.summary()

In [0]:
# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])

results = model.fit(x_teach, np.asarray(y_teach), epochs=2, batch_size=100, validation_data=(x_test, np.asarray(y_test)),
                    verbose=1)



In [0]:
path_file_Desc="VGG_based_descr_256_3_3.hdf5"#przeuczona
model.save(path_file_Desc)

In [0]:
saveToFile = "descriptor_size_256_3_3_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "descriptor_size_256_3_3_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## Dropout

In [0]:
model = Sequential()

model.add(layers.InputLayer((100, 100, 3)))
model.add(layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv1'))
model.add( layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv1'))
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv1'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv2'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv1'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv1'))
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.add(layers.Flatten())
#tutaj zminaić drouput, można by też robić po każdej grupie warstw conv 
model.add(Dropout(0.1)) # mniejsze od 0.1
model.add(layers.Dense(71, activation='softmax'))

model.summary()

In [0]:
# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])

results = model.fit(x_teach, np.asarray(y_teach), epochs=2, batch_size=100, validation_data=(x_test, np.asarray(y_test)),
                    verbose=1)


In [0]:
saveToFile = "robocza_nazwa_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "robocza_nazwa_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## Wczesne zatrzymanie uczenia

In [0]:
model = Sequential()

model.add(layers.InputLayer((100, 100, 3)))
model.add(layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv1'))
model.add( layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv1'))
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv1'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv2'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv1'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv1'))
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.add(layers.Flatten())

model.add(layers.Dense(71, activation='softmax'))

model.summary()


In [0]:
# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])

cb_early = [EarlyStopping(monitor='val_loss', patience=2, min_delta=0.1)]

results = model.fit(x_teach, np.asarray(y_teach), epochs=2, batch_size=100, validation_data=(x_test, np.asarray(y_test)),
                   verbose=1,callbacks= cb_early)
#model.fit(x_teach,np.asarray(y_teach),batch_size=100,epochs=3,validation_data=(x_test, np.asarray(y_test)))

In [0]:
saveToFile = "robocza_nazwa_loss"
plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "robocza_nazwa_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)

## Wpływ przesłonięcia obrazów

In [0]:

model = Sequential()

model.add(layers.InputLayer((100, 100, 3)))
model.add(layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv1'))
model.add( layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv1'))
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv1'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv2'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv1'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv1'))
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.add(layers.Flatten())

model.add(layers.Dense(71, activation='softmax'))

model.summary()


In [0]:
#tu jest parametr do zmiany liczby procentów zaciemnienia obrazu - teraz jest 40 procent
percent = 40
datagennewnew_cover = ImageDataGenerator(fill_mode='nearest')
x_teach_noise = np.copy(x_teach)

for x in range(0, x_teach_noise.shape[0]):
    img = np.reshape(x_teach_noise[x], (100, 100, 3))
    for h in range(img.shape[0]):
        for j in range(img.shape[1]):
            if h < percent:
                img[j, h] = 0

datagennewnew_cover.fit(x_teach_noise)
for X_batch, y_batch in datagennewnew_cover.flow(x_teach_noise, y_teach, batch_size=9):
    for i in range(0, 9):
        pyplot.subplot(330 + 1 + i)
        pyplot.imshow(X_batch[i].reshape(100, 100, 3).astype('uint8'))
    pyplot.show()
    break

In [0]:
# Konfiguracja sieci i danych
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['categorical_accuracy'])


results = model.fit_generator(traind_data_generator, steps_per_epoch=len(x_teach_noise) / 100, epochs=1,validation_data=(x_test, np.asarray(y_test)), verbose=1)


In [0]:
path_file_occulusion="VGG_based_occulusion_60.hdf5"#przeuczona
model.save(path_file_occulusion)

In [0]:
saveToFile = "occulusion_60_loss"
# plt.figure(figsize=[8,6])
plt.plot(results.history['loss'],'r',linewidth=3.0)
plt.plot(results.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Test Loss'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)


In [0]:
saveToFile = "occulusion_60_accuracy"
plt.figure(figsize=[8,6])
plt.plot(results.history['categorical_accuracy'],'r',linewidth=3.0)
plt.plot(results.history['val_categorical_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Test Accuracy'],fontsize=18)
plt.xlabel('Number of epochs ',fontsize=16)
plt.ylabel('Categorical Accuracy',fontsize=16)
plt.title('Categorical Accuracy Plot',fontsize=16)
plt.savefig("wykresy/" + saveToFile)